# Capstone Project - Battle of Neighborhoods
## Live closer to work
Utilizing Foursquare data to recommend place to live in New York

## Katarzyna Cybulska

# Introduction

New Yorkers have the longest commute in USA. Long commute not only deprives you of valuable time but also forces you to stay in crowded, stuffy buses or subway platforms. This project aims as crating a tool, which for given home and work address will recommend neighborhoods closer to the workplace resembling current neighborhood. 

## Import packages
Start with importing required packages.

In [72]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # environment to handle requests for geographical location
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle url requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import math as m # for calculating distance

!conda install -c conda-forge folium=0.5.0 --yes # environment to hadle mapplotting
import folium # map rendering library

import scipy as sp # library contains function, which calculates euclidean distance between point in multidimentional spaces.
from scipy import spatial

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


# Define function to measure distance between geographical points.
For given latitude and longitude of two addresses it is possible to calculate a straight line distance between them. This is a simple geometry task calculating smallest distance on a sphere. I will implement distance calculation based on spherical law of cosines, more info here: https://www.movable-type.co.uk/scripts/latlong.html  

In [73]:
# define geo_dist function which for gived geographical goordinates of two places will return distance between them

def geo_dist(lat1, lon1,lat2, lon2):
        
    # convert degrees to radians
    φ1 = m.radians(lat1)
    φ2 = m.radians(lat2)
    Δλ = m.radians(lon2-lon1)
    R = 6371 # earth radius in meters
    
    dist = round((m.acos( m.sin(φ1)*m.sin(φ2) + m.cos(φ1)*m.cos(φ2) * m.cos(Δλ) ) * R),2)
    
       
    return(dist)

In [74]:
# let's test it on distance between Warsaw and Gliwice (Poland). It should be little less than 300 km.

# initialize geolocator
geolocator = Nominatim(user_agent = "Kasia")

address = 'Gliwice, Poland'

location = geolocator.geocode(address)
lat1 = location.latitude
lon1 = location.longitude

address = 'Warszawa, Poland'

location = geolocator.geocode(address)
lat2 = location.latitude
lon2 = location.longitude

geo_dist(lat1,lon1,lat2,lon2)

272.5

Equipped with function measuring distance between two given addresses we can proceed to exploring and profiling neighborhoods in New York.
# Get New York data
There are tons of free data sets describing New York over the internet. Website http://coredata.nyc/ is definitely a source of inspiration. For my convenience I will use dataset containing New York's boroughs, neighborhoods with their geographical coordinates.

In [75]:
# download data in a json file
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']
# convert json file to pandas data frame

Data downloaded!


Data is now stored in variable newyork_data. Transform data into handy pandas dataframe.

In [76]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# lopp through json file and put required data in dataframe
for data in neighborhoods_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
#display first 5 rows of neighborhoods dataframe
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Visualize data ona map using folium.

In [77]:
# create map of New York using latitude and longitude values

address = "New York, NY"

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
 
    
map_newyork

# Profile New York neighborhoods using Foursquare data

In [78]:
# Define function which returns venues nearby given loaction

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Provide Foursquare credentials, set version and limit.

In [79]:
# provide your Foursquare Client ID and Secret


CLIENT_ID = 'F3D1HU0REAPAONS1WNUE2FN3A5IUNTX2V2CH0HEZC5I3FX2Z' # your Foursquare ID
CLIENT_SECRET = '0NCPT40FOCROE2AS02ELMTZED5DDYRLWSKOH0S4PRNN1W2WR'
VERSION = '20181006' # Foursquare API version
LIMIT = 150

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F3D1HU0REAPAONS1WNUE2FN3A5IUNTX2V2CH0HEZC5I3FX2Z
CLIENT_SECRET:0NCPT40FOCROE2AS02ELMTZED5DDYRLWSKOH0S4PRNN1W2WR


Get venues in the neighborhoods. Function will display names of neighborhoods while running.

In [82]:
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'])

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

View results


In [83]:
newyork_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896521,-73.844680,Pharmacy
2,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898283,-73.850478,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
4,Wakefield,40.894705,-73.847201,Dunkin Donuts,40.890631,-73.849027,Donut Shop


## One hot encoding
We will transform above dataframe using one hot encoding. As a result we will get a dataframe with venue categories as column names. Each row will consist of 0 and only one 1, in a column representing particular venue category.

In [84]:
# one hot encoding
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]

newyork_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store

Groupping by neighborhood  and taking mean of values will compose a vector of characteristics of each neighborhood. 

In [85]:
newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()
newyork_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Store

Merge neighborhood profiles with borough name, londitude and latitude.

In [86]:
newyork_merged = neighborhoods.set_index("Neighborhood").join(newyork_grouped.set_index("Neighborhood")).reset_index()

# in case there were no venues nearby, there would be a row of NaNs. Change it to zeros to aviod error later.
newyork_merged = newyork_merged.fillna(0)
print(newyork_merged.shape)
newyork_merged.head()

(306, 427)


,Neighborhood,Borough,Latitude,Longitude,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor

# Define recommendation function

Function code is rather lengthy. There are few basic steps:
1. get geographical coordinates of home and work address
2. create a profile of current neighborhood using Foursqure Data
3. calculate distance of each New York neighborhood from work place
4. calculate similarity of neighborhoods to home neighborhood
5. generate suggestions

In [87]:
def recommend(home_address, work_address,  newyork_merged_bis, radius=500):


##############################################################
# 1. get geographical coordinates of home and work address
     
    try:
        location = geolocator.geocode(home_address)
        home_lat = location.latitude
        home_lon = location.longitude
    except:
        print("Oops! Check home address.")
        return
        
    try:    
        location = geolocator.geocode(work_address)
        work_lat = location.latitude
        work_lon = location.longitude
    except:
        print("Oops! Check work address.")
        return
    
    current_distance = geo_dist(home_lat, home_lon, work_lat,work_lon)
    print("Your current distance from home to work is {} km.".format(current_distance))
    
##############################################################    
# 2. create a profile of current neighborhood using Foursqure Data
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            home_lat, 
            home_lon, 
            radius, 
            LIMIT)

    name = "Current home neighborhood"
            
# make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
# initilize empty list
    venues_list=[]    
# return only relevant information for each nearby venue
    venues_list.append([(
            name, 
            home_lat, 
            home_lon, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

# put venues into pandas dataframe
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']  

    print("There are {} venues in your neighborhood.".format(nearby_venues.shape[0]))

# one hot encoding
    home_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
    home_onehot['Neighborhood'] = nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
    fixed_columns = [home_onehot.columns[-1]] + list(home_onehot.columns[:-1])
    home_onehot = home_onehot[fixed_columns]

    home_onehot.head()  

    home_grouped = home_onehot.groupby("Neighborhood").mean().reset_index()


# home_grouped is a profile similar to one created for all New York neighborhoods, however it's missing some columns.
# create data set with exact same column names as newyork_merged_bis

    values = [tuple(['home','home',home_lat, home_lon]+[0]*(newyork_merged_bis.shape[1]-4))]
    home_merged = pd.DataFrame.from_records(values,columns = newyork_merged_bis.columns.tolist())



# change values in data frame according to home_grouped

    for i in range(0,home_merged.shape[1]):
        for j in range(0,home_grouped.shape[1]):
            if home_merged.columns.tolist()[i] == home_grouped.columns.tolist()[j]:
                home_merged.iloc[0,i] = home_grouped.iloc[0,j]
    

    print("Your neighborhood has been profiled.")

######################################################
# 3. calculate distance of each New York neighborhood from work place

# initialize empty list
    hood_dist = []

# loop through neighborhoods and calculate distance
    for  lat, lon in zip(newyork_merged_bis['Latitude'], newyork_merged_bis['Longitude']):
        hood_dist = hood_dist + [geo_dist(work_lat,work_lon, lat, lon)]

# insert distance to dataframe
    newyork_merged_bis['Distance'] = hood_dist

##########################################################
# 4. calculate similarity of neighborhoods

#initialize empty list
    profile_dist = []

    home_profile = np.array(home_merged.values.tolist()[0][4::1])
# loop through neighborhood and calculate similarity to home neighborhood
    for i in range(0,newyork_merged_bis.shape[0]):
        hood_profile = np.array(newyork_merged_bis.values.tolist()[i][4:-1])
        profile_dist = profile_dist + [sp.spatial.distance.euclidean(home_profile, hood_profile)]

# add to dataframe
    newyork_merged_bis['similarity'] = profile_dist

###################################################
# 5. generate suggestions

    new_york_zone_1 = newyork_merged_bis.query("Distance < 1").sort_values('similarity')
    new_york_zone_2 = newyork_merged_bis.query("Distance > 1 and Distance <= 3").sort_values('similarity')
    new_york_zone_3 = newyork_merged_bis.query("Distance > 3 and Distance <= 5").sort_values('similarity')

    print("There are {} neighborhoods less then 1 km away from your work.".format(new_york_zone_1.shape[0]))
    if new_york_zone_1.shape[0] > 0:
        print("In this zone we reccomend {}, {}.".format(new_york_zone_1.iloc[0,0],new_york_zone_1.iloc[0,1]))
    print("There are {} neighborhoods between 1 - 3 km away from your work.".format(new_york_zone_2.shape[0]))
    if new_york_zone_2.shape[0] > 0:
        print("In this zone we reccomend {}, {}.".format(new_york_zone_2.iloc[0,0],new_york_zone_2.iloc[0,1]))
    print("There are {} neighborhoods between 3 - 5 km away from your work.".format(new_york_zone_3.shape[0]))
    if new_york_zone_3.shape[0] > 0:
        print("In this zone we reccomend {}, {}.".format(new_york_zone_3.iloc[0,0],new_york_zone_3.iloc[0,1]))
    
# remove added columns
    newyork_merged.drop('Distance', axis = 1, inplace = True)
    newyork_merged.drop('similarity', axis = 1, inplace = True)
    
###############################################
# 
    

## Use recommend() function on example

Let's take as an example Mark, who currently lives in 1030 Neil Avenue, Bronx, NY and commutes every day to work at 545 5th Ave, Manhattan, NY.

In [88]:
home_address = "1030 Neil Avenue, Bronx, NY"
work_address = "545, 5th Ave, Manhattan, NY"

    # get geographocal coordinates of home and work address
    
recommend(home_address, work_address,  newyork_merged)

Your current distance from home to work is 14.29 km.
There are 20 venues in your neighborhood.
Your neighborhood has been profiled.
There are 2 neighborhoods less then 1 km away from your work.
In this zone we reccomend Midtown, Manhattan.
There are 16 neighborhoods between 1 - 3 km away from your work.
In this zone we reccomend Lenox Hill, Manhattan.
There are 20 neighborhoods between 3 - 5 km away from your work.
In this zone we reccomend Sunnyside, Queens.


Let's visualize recommendation on a map. First create data frame holding coordinates of home address, work address and recommended neighborhoods.

In [46]:
# get data for reccomended neighborhoods
for_plotting = neighborhoods.query("(Neighborhood in ('Midtown','Lenox Hill') and Borough == 'Manhattan') or (Neighborhood == 'Sunnyside' and Borough == 'Queens')")

# get home and work address coordinates
location = geolocator.geocode(home_address)
home_lat = location.latitude
home_lon = location.longitude
location = geolocator.geocode(work_address)
work_lat = location.latitude
work_lon = location.longitude
    
for_plotting.head()


,Borough,Neighborhood,Latitude,Longitude
109,Manhattan,Lenox Hill,40.768113,-73.958860
114,Manhattan,Midtown,40.754691,-73.981669
140,Queens,Sunnyside,40.740176,-73.926916


Create a map

In [68]:
# create map of New York using latitude and longitude values

address = "New York, NY"

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

recommendation_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add reccomendations in blue
for lat, lng,borough, neighborhood in zip(for_plotting['Latitude'], for_plotting['Longitude'],for_plotting['Borough'], for_plotting['Neighborhood']):
    label = '{},{}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=False,
        parse_html=False).add_to(recommendation_map)  
    
# add home as yellow

label = '{}'.format('home')
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
        [home_lat, home_lon],
        radius=4,
        popup=label,
        color='yellow',
        fill=False,
        parse_html=False).add_to(recommendation_map)  



# add work in red

label = '{}'.format('work')
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
        [work_lat, work_lon],
        radius=4,
        popup=label,
        color='red',
        fill=False,
        parse_html=False).add_to(recommendation_map)

   
recommendation_map

# Done!

## Thanks for reading!